<a href="https://colab.research.google.com/github/davidecopp/Model-Serie-A-2021-22/blob/main/SerieA_vsc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import statistics as stat
import matplotlib.pyplot as plt

In [ ]:
campionato = pd.read_csv('campionato_aggiornato26giornata.csv').drop('CRISI',axis = 1)
squadre = sorted(campionato['CASA'].unique().tolist())
#Squadre = pd.DataFrame(columns = ['Squadre'])
#Squadre['Squadre'] = squadre

In [ ]:
x_plus_y = (campionato['X']+campionato['Y'])[0:180]
x_minus_y = (campionato['X']-campionato['Y'])[0:180]
S = np.zeros((180,21))
T = np.zeros((180,21))

for i in range(180):
  S[i][20] = 1
  T[i][20] = 1
  for j in range(20):
    if campionato['CASA'][i] == squadre[j]:
      S[i][j] = 1
      T[i][j] = 1
    if campionato['TRASFERTA'][i] == squadre[j]:
      S[i][j] = 1
      T[i][j] = -1

In [ ]:
alfa = np.linalg.lstsq(S, x_plus_y, rcond = None)[0]
beta = np.linalg.lstsq(T, x_minus_y, rcond = None)[0]

lambdas_matrix = np.zeros((20,20,2))
for i in range(20):
  for j in range(20):
    if i != j:
      a = (alfa[i]+alfa[j]+alfa[20]+(beta[i]-beta[j]+beta[20]))/2
      b = (alfa[i]+alfa[j]+alfa[20]-(beta[i]-beta[j]+beta[20]))/2
      lambdas_matrix[i][j][0] = a
      lambdas_matrix[i][j][1] = b
      if a < 0:
        lambdas_matrix[i][j][0] = 0.25
      if b < 0:
        lambdas_matrix[i][j][1] = 0.25

In [ ]:
Classifica_iniziale = pd.DataFrame(np.zeros((20,8)),columns = ['PG','V','P','S','GF','GS', 'DR','Punti'], index = squadre)  
for i in range(180):

  Classifica_iniziale['GF'][campionato['CASA'][i]] += campionato['X'][i]
  Classifica_iniziale['GF'][campionato['TRASFERTA'][i]] += campionato['Y'][i]
  Classifica_iniziale['GS'][campionato['CASA'][i]] += campionato['Y'][i]
  Classifica_iniziale['GS'][campionato['TRASFERTA'][i]] += campionato['X'][i]
  Classifica_iniziale['DR'][campionato['CASA'][i]] += campionato['X'][i]-campionato['Y'][i]
  Classifica_iniziale['DR'][campionato['TRASFERTA'][i]] += campionato['Y'][i]-campionato['X'][i]
  Classifica_iniziale['PG'][campionato['CASA'][i]] += 1
  Classifica_iniziale['PG'][campionato['TRASFERTA'][i]] += 1

  if campionato['X'][i] > campionato['Y'][i]:
    Classifica_iniziale['V'][campionato['CASA'][i]] += 1
    Classifica_iniziale['S'][campionato['TRASFERTA'][i]] += 1
    Classifica_iniziale['Punti'][campionato['CASA'][i]] += 3

  elif campionato['X'][i] < campionato['Y'][i]:
    Classifica_iniziale['V'][campionato['TRASFERTA'][i]] += 1
    Classifica_iniziale['S'][campionato['CASA'][i]] += 1
    Classifica_iniziale['Punti'][campionato['TRASFERTA'][i]] += 3 

  else:
    Classifica_iniziale['P'][campionato['CASA'][i]] += 1
    Classifica_iniziale['P'][campionato['TRASFERTA'][i]] += 1
    Classifica_iniziale['Punti'][campionato['CASA'][i]] += 1
    Classifica_iniziale['Punti'][campionato['TRASFERTA'][i]] += 1

Classifica_iniziale.sort_values(by = ['Punti','DR'], ascending = False).reset_index()


,index,PG,V,P,S,GF,GS,DR,Punti
0,Inter,18.0,13.0,4.0,1.0,48.0,15.0,33.0,43.0
1,Napoli,18.0,12.0,3.0,3.0,35.0,13.0,22.0,39.0
2,Milan,18.0,12.0,3.0,3.0,36.0,20.0,16.0,39.0
3,Atalanta,18.0,11.0,4.0,3.0,38.0,24.0,14.0,37.0
4,Roma,18.0,10.0,1.0,7.0,30.0,20.0,10.0,31.0
5,Fiorentina,18.0,10.0,1.0,7.0,33.0,24.0,9.0,31.0
6,Juventus,18.0,9.0,4.0,5.0,25.0,17.0,8.0,31.0
7,Lazio,18.0,8.0,4.0,6.0,36.0,33.0,3.0,28.0
8,Empoli,18.0,8.0,3.0,7.0,28.0,30.0,-2.0,27.0
9,Torino,18.0,7.0,4.0,7.0,23.0,18.0,5.0,25.0


In [ ]:
#SIMULAZIONE RESTANTI PARTITE DI CAMPIONATO
Classifica = Classifica_iniziale.copy()

for k in range(20):
  x_plus_y = (campionato['X']+campionato['Y'])[0:180+10*k]
  x_minus_y = (campionato['X']-campionato['Y'])[0:180+10*k]
  S = np.zeros((180+10*k,21))
  T = np.zeros((180+10*k,21))
  
  for i in range(180+10*k):
    S[i][20] = 1
    T[i][20] = 1
    for j in range(20):
      if campionato['CASA'][i] == squadre[j]:
        S[i][j] = 1
        T[i][j] = 1
      if campionato['TRASFERTA'][i] == squadre[j]:
        S[i][j] = 1
        T[i][j] = -1

  alfa = np.linalg.lstsq(S, x_plus_y, rcond = None)[0]
  beta = np.linalg.lstsq(T, x_minus_y, rcond = None)[0]

  lambdas_matrix = np.zeros((20,20,2))
  for i in range(20):
    for j in range(20):
      if i != j:
        a = (alfa[i]+alfa[j]+alfa[20]+(beta[i]-beta[j]+beta[20]))/2
        b = (alfa[i]+alfa[j]+alfa[20]-(beta[i]-beta[j]+beta[20]))/2
        lambdas_matrix[i][j][0] = a
        lambdas_matrix[i][j][1] = b
        if a < 0:
          lambdas_matrix[i][j][0] = 0.25
        if b < 0:
          lambdas_matrix[i][j][1] = 0.25

  for i in range(180+10*k,180+10*(k+1)):
    campionato['X'][i] = np.random.poisson(lam = lambdas_matrix[squadre.index(campionato['CASA'][i])][squadre.index(campionato['TRASFERTA'][i])][0])
    campionato['Y'][i] = np.random.poisson(lam = lambdas_matrix[squadre.index(campionato['TRASFERTA'][i])][squadre.index(campionato['CASA'][i])][1])

    Classifica['GF'][campionato['CASA'][i]] += campionato['X'][i]
    Classifica['GF'][campionato['TRASFERTA'][i]] += campionato['Y'][i]
    Classifica['GS'][campionato['CASA'][i]] += campionato['Y'][i]
    Classifica['GS'][campionato['TRASFERTA'][i]] += campionato['X'][i]
    Classifica['DR'][campionato['CASA'][i]] += campionato['X'][i]-campionato['Y'][i]
    Classifica['DR'][campionato['TRASFERTA'][i]] += campionato['Y'][i]-campionato['X'][i]
    Classifica['PG'][campionato['CASA'][i]] += 1
    Classifica['PG'][campionato['TRASFERTA'][i]] += 1

    if campionato['X'][i]>campionato['Y'][i]: #VITTORIA SQUADRA DI CASA
      Classifica['V'][campionato['CASA'][i]] += 1
      Classifica['S'][campionato['TRASFERTA'][i]] += 1
      Classifica['Punti'][campionato['CASA'][i]] += 3

    elif campionato['X'][i]<campionato['Y'][i]: #SCONFITTA SQUADRA DI CASA
      Classifica['V'][campionato['TRASFERTA'][i]] += 1
      Classifica['S'][campionato['CASA'][i]] += 1
      Classifica['Punti'][campionato['TRASFERTA'][i]] += 3 

    else: #PAREGGIO
      Classifica['P'][campionato['CASA'][i]] += 1
      Classifica['P'][campionato['TRASFERTA'][i]] += 1
      Classifica['Punti'][campionato['CASA'][i]] += 1
      Classifica['Punti'][campionato['TRASFERTA'][i]] += 1
      
Classifica.sort_values(by = ['Punti','DR'], ascending = False).reset_index()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,PG,V,P,S,GF,GS,DR,Punti
0,Milan,38.0,22.0,7.0,9.0,87.0,56.0,31.0,73.0
1,Inter,38.0,21.0,7.0,10.0,93.0,62.0,31.0,70.0
2,Fiorentina,38.0,22.0,4.0,12.0,75.0,56.0,19.0,70.0
3,Atalanta,38.0,19.0,8.0,11.0,70.0,53.0,17.0,65.0
4,Lazio,38.0,19.0,7.0,12.0,96.0,75.0,21.0,64.0
5,Empoli,38.0,19.0,7.0,12.0,71.0,57.0,14.0,64.0
6,Roma,38.0,19.0,3.0,16.0,66.0,49.0,17.0,60.0
7,Napoli,38.0,16.0,11.0,11.0,59.0,45.0,14.0,59.0
8,Juventus,38.0,15.0,13.0,10.0,42.0,31.0,11.0,58.0
9,Torino,38.0,17.0,7.0,14.0,53.0,47.0,6.0,58.0


In [ ]:
# N simulazioni del campionato 
N = 100
Punti = pd.DataFrame(np.zeros((N,20)), columns = squadre)
Risultato = pd.DataFrame(np.zeros((N,20)), columns = squadre)

for n in range(N):
  Classifica = Classifica_iniziale.copy()

  for k in range(20):
    x_plus_y = (campionato['X']+campionato['Y'])[0:180+10*k]
    x_minus_y = (campionato['X']-campionato['Y'])[0:180+10*k]
    S = np.zeros((180+10*k,21))
    T = np.zeros((180+10*k,21))
    
    for i in range(180+10*k):
      S[i][20] = 1
      T[i][20] = 1
      for j in range(20):
        if campionato['CASA'][i] == squadre[j]:
          S[i][j] = 1
          T[i][j] = 1
        if campionato['TRASFERTA'][i] == squadre[j]:
          S[i][j] = 1
          T[i][j] = -1

    alfa = np.linalg.lstsq(S, x_plus_y, rcond = None)[0]
    beta = np.linalg.lstsq(T, x_minus_y, rcond = None)[0]

    lambdas_matrix = np.zeros((20,20,2))
    for i in range(20):
      for j in range(20):
        if i != j:
          a = (alfa[i]+alfa[j]+alfa[20]+(beta[i]-beta[j]+beta[20]))/2
          b = (alfa[i]+alfa[j]+alfa[20]-(beta[i]-beta[j]+beta[20]))/2
          lambdas_matrix[i][j][0] = a
          lambdas_matrix[i][j][1] = b
          if a < 0:
            lambdas_matrix[i][j][0] = 0.25
          if b < 0:
            lambdas_matrix[i][j][1] = 0.25

    for i in range(180+10*k,180+10*(k+1)):
      campionato['X'][i] = np.random.poisson(lam = lambdas_matrix[squadre.index(campionato['CASA'][i])][squadre.index(campionato['TRASFERTA'][i])][0])
      campionato['Y'][i] = np.random.poisson(lam = lambdas_matrix[squadre.index(campionato['TRASFERTA'][i])][squadre.index(campionato['CASA'][i])][1])

      Classifica['GF'][campionato['CASA'][i]] += campionato['X'][i]
      Classifica['GF'][campionato['TRASFERTA'][i]] += campionato['Y'][i]
      Classifica['GS'][campionato['CASA'][i]] += campionato['Y'][i]
      Classifica['GS'][campionato['TRASFERTA'][i]] += campionato['X'][i]
      Classifica['DR'][campionato['CASA'][i]] += campionato['X'][i]-campionato['Y'][i]
      Classifica['DR'][campionato['TRASFERTA'][i]] += campionato['Y'][i]-campionato['X'][i]
      Classifica['PG'][campionato['CASA'][i]] += 1
      Classifica['PG'][campionato['TRASFERTA'][i]] += 1

      if campionato['X'][i] > campionato['Y'][i]: #VITTORIA SQUADRA DI CASA
        Classifica['V'][campionato['CASA'][i]] += 1
        Classifica['S'][campionato['TRASFERTA'][i]] += 1
        Classifica['Punti'][campionato['CASA'][i]] += 3

      elif campionato['X'][i] < campionato['Y'][i]: #SCONFITTA SQUADRA DI CASA
        Classifica['V'][campionato['TRASFERTA'][i]] += 1
        Classifica['S'][campionato['CASA'][i]] += 1
        Classifica['Punti'][campionato['TRASFERTA'][i]] += 3 

      else: #PAREGGIO
        Classifica['P'][campionato['CASA'][i]] += 1
        Classifica['P'][campionato['TRASFERTA'][i]] += 1
        Classifica['Punti'][campionato['CASA'][i]] += 1
        Classifica['Punti'][campionato['TRASFERTA'][i]] += 1
        
  Classifica = Classifica.sort_values(by = ['Punti','DR'], ascending = False).reset_index()

  posizione = 1
  for team in Classifica['index']:
    Risultato[team][n] = posizione
    Punti[team][n] = Classifica['Punti'][posizione-1]
    posizione += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
Risultato

,Atalanta,Bologna,Cagliari,Empoli,Fiorentina,Genoa,Inter,Juventus,Lazio,Milan,Napoli,Roma,Salernitana,Sampdoria,Sassuolo,Spezia,Torino,Udinese,Venezia,Verona
0,4.0,9.0,16.0,7.0,10.0,19.0,1.0,5.0,11.0,3.0,2.0,13.0,17.0,18.0,6.0,20.0,8.0,12.0,14.0,15.0
1,3.0,13.0,17.0,11.0,7.0,19.0,5.0,6.0,12.0,2.0,1.0,8.0,20.0,14.0,4.0,18.0,9.0,16.0,15.0,10.0
2,2.0,9.0,15.0,8.0,6.0,18.0,1.0,12.0,10.0,3.0,5.0,7.0,19.0,13.0,16.0,20.0,11.0,14.0,17.0,4.0
3,3.0,17.0,18.0,7.0,4.0,19.0,1.0,13.0,16.0,2.0,5.0,12.0,20.0,6.0,11.0,15.0,8.0,10.0,14.0,9.0
4,5.0,11.0,16.0,9.0,6.0,15.0,1.0,4.0,8.0,3.0,2.0,10.0,19.0,18.0,7.0,20.0,14.0,13.0,17.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4.0,11.0,20.0,6.0,5.0,19.0,1.0,9.0,13.0,10.0,7.0,2.0,18.0,12.0,3.0,17.0,8.0,16.0,14.0,15.0
96,2.0,6.0,20.0,14.0,10.0,18.0,1.0,11.0,13.0,3.0,4.0,7.0,19.0,16.0,12.0,17.0,8.0,5.0,15.0,9.0
97,4.0,7.0,18.0,11.0,6.0,16.0,2.0,1.0,10.0,5.0,3.0,9.0,20.0,17.0,14.0,19.0,8.0,13.0,15.0,12.0
98,4.0,13.0,18.0,6.0,5.0,20.0,3.0,8.0,9.0,7.0,1.0,10.0,19.0,17.0,11.0,16.0,2.0,15.0,12.0,14.0


In [ ]:
Punti

,Atalanta,Bologna,Cagliari,Empoli,Fiorentina,Genoa,Inter,Juventus,Lazio,Milan,Napoli,Roma,Salernitana,Sampdoria,Sassuolo,Spezia,Torino,Udinese,Venezia,Verona
0,63.0,53.0,44.0,54.0,52.0,37.0,72.0,61.0,52.0,64.0,68.0,50.0,44.0,43.0,56.0,34.0,53.0,50.0,50.0,49.0
1,66.0,48.0,43.0,51.0,60.0,39.0,61.0,60.0,50.0,66.0,67.0,56.0,30.0,47.0,65.0,41.0,52.0,45.0,47.0,51.0
2,67.0,57.0,44.0,59.0,61.0,38.0,73.0,49.0,56.0,63.0,62.0,59.0,33.0,48.0,42.0,30.0,50.0,47.0,42.0,62.0
3,68.0,44.0,42.0,56.0,62.0,33.0,74.0,50.0,45.0,69.0,59.0,50.0,30.0,58.0,51.0,45.0,55.0,53.0,48.0,54.0
4,62.0,56.0,38.0,59.0,62.0,40.0,70.0,65.0,60.0,70.0,70.0,58.0,31.0,36.0,61.0,30.0,47.0,47.0,38.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,64.0,53.0,25.0,62.0,64.0,30.0,71.0,59.0,47.0,59.0,60.0,71.0,36.0,49.0,68.0,38.0,59.0,44.0,45.0,45.0
96,71.0,60.0,30.0,47.0,53.0,40.0,78.0,53.0,47.0,64.0,62.0,58.0,38.0,42.0,49.0,42.0,57.0,60.0,43.0,54.0
97,68.0,56.0,33.0,52.0,64.0,39.0,71.0,77.0,53.0,66.0,70.0,55.0,30.0,33.0,44.0,33.0,55.0,50.0,41.0,51.0
98,68.0,48.0,34.0,64.0,65.0,31.0,69.0,59.0,54.0,61.0,71.0,54.0,34.0,38.0,49.0,41.0,71.0,46.0,49.0,46.0


In [ ]:
Outcome=pd.DataFrame(columns=['Punti medi','Scudetto','Champions League','Serie B'], index=squadre, dtype=float)
for team in Risultato:
  Outcome['Punti medi'][team]=Punti[team].sum()/N
  Outcome['Scudetto'][team]=Risultato[team][Risultato[team]==1].value_counts().sum()/N
  Outcome['Champions League'][team]=Risultato[team][Risultato[team]<=4].value_counts().sum()/N
  Outcome['Serie B'][team]=Risultato[team][Risultato[team]>=18].value_counts().sum()/N
Outcome

,Punti medi,Scudetto,Champions League,Serie B
Atalanta,65.34,0.11,0.72,0.00
Bologna,52.13,0.00,0.04,0.04
Cagliari,37.03,0.00,0.00,0.73
Empoli,55.28,0.01,0.12,0.00
Fiorentina,58.24,0.03,0.18,0.00
Genoa,38.05,0.00,0.00,0.60
Inter,70.05,0.43,0.89,0.00
Juventus,58.19,0.03,0.18,0.00
Lazio,55.94,0.02,0.12,0.01
Milan,66.48,0.17,0.77,0.00
